In [ ]:
import os
import pickle
from scipy.stats import gaussian_kde
from sklearn.neighbors import KernelDensity

In [ ]:
from srcfanova.confspace_utils import get_configspace, integer_encode_dataframe


import itertools as it
from collections import OrderedDict

import ConfigSpace
import numpy as np
import pandas as pd
import pyrfr.regression as reg
import pyrfr.util
from ConfigSpace.hyperparameters import CategoricalHyperparameter, UniformFloatHyperparameter, \
    NumericalHyperparameter, Constant, OrdinalHyperparameter

from surrogate import fANOVA_surrogate
from hyperband_finite import HyperBandOptimiser

from typing import List

# Prepare data for fitting surrogate models 

In [ ]:
dataf = pd.read_csv("./nonaggresults_hyper.csv", sep=",").groupby(by=['batchsize',
 'depth',
 'entangler_operation',
 'have_less_rotations',
 'input_activation_function',
 'is_data_encoding_hardware_efficient',
 'learning_rate',
 'map_type',
 'output_circuit',
 'use_reuploading',
'dataset', "task_id", "epochs"]).agg({'val_binary_accuracy': 'max'}).reset_index()

# removing these two datasets because the performance can not be explained using ANOVA
dataf = dataf[dataf.dataset != 'ilpd']
dataf = dataf[dataf.dataset != 'blood-transfusion-service-center']
# del dataf['
dataf = dataf.reset_index()
task_ids = sorted(dataf['task_id'].unique())

measure = 'val_binary_accuracy'

# important hyperparameter keys to consider: learning_rate, depth, use_reuploading, input_activation_function
# make sure data is numerical and in right order for configspace
config_space = get_configspace(bool(1))
cs_params = config_space.get_hyperparameter_names()

original_df = dataf.loc[:, [cs_params[i] for i in range(len(cs_params))]]

data = dataf.loc[:, [cs_params[i] for i in range(len(cs_params))]]
data = integer_encode_dataframe(data, config_space)
data['task_id'] = dataf.task_id
data['dataset'] = dataf.dataset
data[measure] = dataf[measure]

# Fit surrogate models

In [ ]:
model_per_task = {}
n_trees= 128

for t_idx, task_id in enumerate(task_ids):
    
    data_task = data[data['task_id'] == task_id]
    del data_task['task_id']
    del data_task['dataset']
    
    y_data = data_task[measure].values
    X_data = data_task.copy()
    del X_data[measure]
    
    model_per_task[task_id] = fANOVA_surrogate(X=X_data, Y=y_data, n_trees=n_trees, seed=t_idx)

In [ ]:
model_per_task

# Hyperband Optimisation Uniform vs KDE Prior Experiment

The experiment configuration for the uniform prior experiment is defined in the next cell.

In [ ]:
n_runs = 15

seed_exp = np.arange(n_runs)
eta_exp = [2, 3, 4] # halving factor
max_iter_exp = [int(50), int(1e2)] # epochs, can be though of as number of shots to compute the expectation value.
search_type = 'uniform' 

# Run hyperband algorithm with uniform priors

Doing a search over hyperparameter configuration space by employing hyperband with uniform sampling of hyperparameters. The objective is to find the hyperparameter configuration which gives maximum validation accuracy of the surrogate model.

In [ ]:
# For uniform prior experiments

for max_iter in max_iter_exp:
    for eta in eta_exp:
        for task_id in task_ids:
            for seed in seed_exp:
                optimiser = HyperBandOptimiser(eta=eta,
                                                       config_space=config_space,
                                                       optimisation_goal='performance',
                                                       max_iter=max_iter,
                                                       min_or_max=max,
                                                       task_id=task_id,
                                                       search_type='uniform',
                                                       seed_nb=seed,
                                                       pickle_path=None)

                best_config = optimiser.run_optimisation(model_for_task_id=model_per_task[task_id],
                                                                 all_data=data,
                                                                 store_optimiser=True,
                                                                 verbosity=False)
                # print(best_config)

# Retrieve & Save necessary results  

In [ ]:

for max_iter in max_iter_exp:
    for eta in eta_exp:
        
        results = {}

        for task_id in task_ids:
            results[task_id] = {}

        for task_id in task_ids:
            for seed in seed_exp:
                f_name = f'./optimiser/{search_type}/task_id{task_id}_search_{search_type}_eta_{eta}_max_iter_{max_iter}_seed_{seed}.pckl'
                optimiser = pickle.load(open(f_name, 'rb'))
                # results[task_id][seed] = optimiser.eval_history
                results[task_id][seed] = {'eval_history': optimiser.eval_history, 'best_config_history': optimiser.config_history_w_perf}

                f_name = f'./data/{search_type}_eta_{eta}_max_iter_{max_iter}.pckl'
                os.makedirs(os.path.dirname(f_name), exist_ok=True)
                pickle.dump(results, open(f_name, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
                    

# Run hyperband algorithm with KDE priors

The experiment configuration for the uniform prior experiment is defined in the next cell.

In [ ]:
n_runs = 15

best_N_exp = [10, 20]
seed_exp = np.arange(n_runs)
eta_exp = [2, 3, 4]
max_iter_exp = [int(50), int(1e2)] 
search_type = 'kde'

# this is given as a list of a list, where the inside list contains the indices of (important) hyperparameters
# Index {6: learning_rate, 1: depth, 4: input_activation_function, 9: use_reuploading}
# Important hyperparams considered in this study are top-K where K \in \{1, 2, 3, 4\}
imp_hyperparams_list_exp = [[6], [6, 1], [6, 1, 4], [6, 1, 4, 9]]
kde_bw_estimator_exp = ['sj', 'silverman'] # bandwith estimator for kde to fit the data

Doing a search over hyperparameter configuration space by employing hyperband with (some, mostly important ones) hyperparameters sampled from kernel density estimator which is fitted with best_N performing hyperparameter configurations for each task.

In [ ]:
# For kde prior experiments

for imp_hyperparams_list in imp_hyperparams_list_exp:
    for kde_bw_estimator in kde_bw_estimator_exp:
        for max_iter in max_iter_exp:
            for eta in eta_exp:
                for best_N in best_N_exp:
                    for task_id in task_ids:
                        for seed in seed_exp:
                            optimiser = HyperBandOptimiser(eta=eta,
                                                           config_space=config_space,
                                                           optimisation_goal='performance',
                                                           max_iter=max_iter,
                                                           min_or_max=max,
                                                           task_id=task_id,
                                                           search_type='kde',
                                                           important_hyperparams_indices=imp_hyperparams_list,
                                                           best_N=best_N,
                                                           seed_nb=seed,
                                                           kde_bw_estimator=kde_bw_estimator,
                                                           kde_bw=None,
                                                           pickle_path=None)

                            best_config = optimiser.run_optimisation(model_for_task_id=model_per_task[task_id],
                                                                     all_data=data,
                                                                     store_optimiser=True,
                                                                     verbosity=False)
                            # print(best_config)

# Retrieve & Save necessary results  

In [ ]:

for imp_hyperparams_list in imp_hyperparams_list_exp:
    for max_iter in max_iter_exp:
        for eta in eta_exp:
            for best_N in best_N_exp:
                for kde_bw_estimator in kde_bw_estimator_exp:

                    results = {}
                    for task_id in task_ids:
                        results[task_id] = {}

                    for task_id in task_ids:
                        for seed in seed_exp:
                            opt_f_name = f'./optimiser/{search_type}/task_id{task_id}_search_{search_type}_bw_None_bw_est_{kde_bw_estimator}_bestN_{best_N}_eta_{eta}_max_iter_{max_iter}_imp_hyp_{imp_hyperparams_list}_seed_{seed}.pckl'
                            optimiser = pickle.load(open(opt_f_name, 'rb'))
                            results[task_id][seed] = {'eval_history': optimiser.eval_history, 'best_config_history': optimiser.config_history_w_perf}
                    
                    f_name = f'./data/{search_type}_bw_est_{kde_bw_estimator}_bestN_{best_N}_eta_{eta}_max_iter_{max_iter}_imp_hyp_{imp_hyperparams_list}.pckl'
                    os.makedirs(os.path.dirname(f_name), exist_ok=True)
                    pickle.dump(results, open(f_name, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
                        

# Data Preparation for the Plots

In [ ]:
data_directory = './data/'
df = pd.DataFrame(columns=['task_id', 'seed', 'max_iter', 'eta', 'imp_hyperparams', 'best_N', 'bw_estimator', 'result_kde', 'result_uniform', 'epochs_kde', 'epochs_uniform', 'difference'])

In [ ]:
def get_max_perf_epochs(results_file, task_id, seed):
    _max_perf = 0
    _epochs = 0
    epoch_history = []
    for key in results_file[task_id][seed]['best_config_history'].keys():
        if len(results_file[task_id][seed]['best_config_history'][key]['performance']) > 1:
            epoch_idx = results_file[task_id][seed]['best_config_history'][key]['epochs'].argmin()
            perf = float(results_file[task_id][seed]['best_config_history'][key]['performance'].values[epoch_idx])
            epoch = int(results_file[task_id][seed]['best_config_history'][key]['epochs'].values[epoch_idx])
        else:
            perf = float(results_file[task_id][seed]['best_config_history'][key]['performance'])
            epoch = int(results_file[task_id][seed]['best_config_history'][key]['epochs'])
        epoch_history.append(epoch)
        if perf > _max_perf:
            _max_perf = perf
            _epochs = epoch
    return _epochs, epoch_history

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

data_plot = []

for imp_hyperparams_list in imp_hyperparams_list_exp:
    for max_iter in max_iter_exp:
        for eta in eta_exp:
            
            f_uni_name = data_directory + f'uniform_eta_{eta}_max_iter_{max_iter}.pckl'
            uni_results = pickle.load(open(f_uni_name, 'rb'))

            for best_N in best_N_exp:
                for kde_bw_estimator in kde_bw_estimator_exp:
                    # data = []                                
                    f_kde_name = data_directory + f'kde_bw_est_{kde_bw_estimator}_bestN_{best_N}_eta_{eta}_max_iter_{max_iter}_imp_hyp_{imp_hyperparams_list}.pckl'
                    kde_results = pickle.load(open(f_kde_name, 'rb'))

                    for task_id in task_ids:
                        for seed in seed_exp:

                            scores_kde = np.max(kde_results[task_id][seed]['eval_history'])
                            scores_uniform = np.max(uni_results[task_id][seed]['eval_history'])
                            
                            epochs_kde, _ = get_max_perf_epochs(kde_results, task_id, seed)
                            epochs_uniform, _ = get_max_perf_epochs(uni_results, task_id, seed)
                            
                            current_difference = scores_kde - scores_uniform
                            # rel_current_difference = (scores_kde - scores_uniform) / scores_uniform
                            data_plot.append(rel_current_difference)

                            current_row = {'task_id': task_id, 'seed': seed, 'max_iter': max_iter, 'eta': eta, 'imp_hyperparams': imp_hyperparams_list, 'best_N': best_N, 'bw_estimator': kde_bw_estimator, 'result_kde': scores_kde, 'result_uniform': scores_uniform, 'epochs_kde': epochs_kde, 'epochs_uniform': epochs_uniform, 'difference': current_difference}
                            df = df.append(current_row, ignore_index=True)
                        

In [ ]:
# For saving the data so that one does not have to redo the post-processing again.
# df.to_csv('kde_vs_uniform_all_data.csv')

# Statistics of Hyperband runs

In [ ]:
for col in df.columns[:-1]:
    df[f'{col}'] = df[f'{col}'].astype('string')
df['difference'] = df['difference'].astype('float')

df_mean = df.groupby(by=['max_iter', 'eta', 'imp_hyperparams', 'best_N', 
                     'bw_estimator'])['difference'].mean().reset_index()



In [ ]:
print(np.mean(data_plot), np.median(data_plot), np.max(data_plot)) # statistics for all runs!

In [ ]:
positive_count = 0
negative_count = 0

for i in data_plot:
    if i > 0:
        positive_count += 1
    else:
        negative_count += 1

print(f'Percentage of positive is {positive_count/len(data_plot)} and negative is {negative_count/len(data_plot)}')

## Best Hyperband run on Average

In [ ]:
df_mean.iloc[df_mean['difference'].idxmax()] # best mean run of all hyperband runs.

## Worst Hyperband run on Average


In [ ]:
df_mean.iloc[df_mean['difference'].idxmin()] # worst mean run of all hyperband runs.

Best run on average stats

In [ ]:
# data for best mean run

n_runs = 15
seed_exp = np.arange(n_runs)
max_iter_best = 50
eta_best = 4
imp_hyperparam_best = [6, 1, 4]
bestN_best = 20
bw_best = 'sj'

f_name_uni_best = f'./data/uniform_eta_{eta_best}_max_iter_{max_iter_best}.pckl'
uni_results_best = pickle.load(open(f_name_uni_best, 'rb'))

f_name_kde_best = f'./data/kde_bw_est_{bw_best}_bestN_{bestN_best}_eta_{eta_best}_max_iter_{max_iter_best}_imp_hyp_{imp_hyperparam_best}.pckl'
kde_results_best = pickle.load(open(f_name_kde_best, 'rb'))

data_plot_best_run = []
for task_id in task_ids:
    for seed in seed_exp:

        scores_kde = np.max(kde_results_best[task_id][seed]['eval_history'])
        scores_uniform = np.max(uni_results_best[task_id][seed]['eval_history'])
        current_difference = scores_kde - scores_uniform
        data_plot_best_run.append(current_difference)

In [ ]:
print(np.mean(data_plot_best_run), np.median(data_plot_best_run), np.max(data_plot_best_run)) # statistics for best run!

In [ ]:
positive_count = 0
negative_count = 0

for i in data_plot_best_run:
    if i > 0:
        positive_count += 1
    else:
        negative_count += 1

print(f'Percentage of positive is {positive_count/len(data_plot_best_run)} and negative is {negative_count/len(data_plot_best_run)}')

Worst run on average stats

In [ ]:
# data for worst mean run

n_runs = 15
seed_exp = np.arange(n_runs)
max_iter_best = 100
eta_best = 3
imp_hyperparam_best = [6, 1]
bestN_best = 20
bw_best = 'sj'


f_name_uni_best = f'./data/uniform_eta_{eta_best}_max_iter_{max_iter_best}.pckl'
uni_results_best = pickle.load(open(f_name_uni_best, 'rb'))

f_name_kde_best = f'./data/kde_bw_est_{bw_best}_bestN_{bestN_best}_eta_{eta_best}_max_iter_{max_iter_best}_imp_hyp_{imp_hyperparam_best}.pckl'
kde_results_best = pickle.load(open(f_name_kde_best, 'rb'))

data_plot_worst_run = []
for task_id in task_ids:
    for seed in seed_exp:

        scores_kde = np.max(kde_results_best[task_id][seed]['eval_history'])
        scores_uniform = np.max(uni_results_best[task_id][seed]['eval_history'])
        current_difference = scores_kde - scores_uniform
        data_plot_worst_run.append(current_difference)

In [ ]:
print(np.mean(data_plot_worst_run), np.median(data_plot_worst_run), np.max(data_plot_worst_run)) # statistics for worst run!

In [ ]:
positive_count = 0
negative_count = 0

for i in data_plot_worst_run:
    if i > 0:
        positive_count += 1
    else:
        negative_count += 1

print(f'Percentage of positive is {positive_count/len(data_plot_worst_run)} and negative is {negative_count/len(data_plot_worst_run)}')

# Plot Helping functions

In [ ]:
def get_violin_plot(data, ax, title, f1, f2):
    
    def draw_quartiles(self, ax, data, support, density, center, split=False):
        mean_ = np.mean(data)
        self.draw_to_density(
            ax,
            center,
            mean_,
            support,
            density,
            split,
            linewidth=self.linewidth,
        )
    sns.set_theme()
    sns.categorical._ViolinPlotter.draw_quartiles = draw_quartiles
    ax.axhline(y=0, color="black", linestyle="--", linewidth=0.6)

    sns.violinplot(data=data, bw='silverman', ax=ax, saturation=0.6, scale='width', cut=0, inner='quartile', linewidth=1.2)
    ax.tick_params(axis='both', which='major', labelsize=10)
    ax.tick_params(axis='both', which='minor', labelsize=8)
    ax.get_xaxis().set_ticks([])
    ax.set_title(f'{title}', fontsize =f1)
    ax.set_ylabel('Improvement', fontsize =f2)
    ax.get_xaxis().set_ticks([])
    
    
def get_violin_plot1(data, ax, title, f1, f2):
    
    def draw_quartiles(self, ax, data, support, density, center, split=False):
        mean_ = np.mean(data)
        self.draw_to_density(
            ax,
            center,
            mean_,
            support,
            density,
            split,
            linewidth=self.linewidth,
        )
    sns.set_theme()
    sns.categorical._ViolinPlotter.draw_quartiles = draw_quartiles
    ax.axhline(y=0, color="black", linestyle="--", linewidth=0.6)

    sns.violinplot(data=data, bw='silverman', ax=ax, saturation=0.6, scale='width', cut=0, inner='quartile', linewidth=1.2)
    ax.tick_params(axis='both', which='major', labelsize=10)
    ax.tick_params(axis='both', which='minor', labelsize=8)
    ax.get_xaxis().set_ticks([])
    ax.set_title(f'{title}', fontsize =f1)
    ax.get_xaxis().set_ticks([])

Figure 8 | Violin Plot for All Hyperband runs, Best and Worst run

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.transforms as mtransforms

fig, axs = plt.subplot_mosaic([['(a)', '(b)', '(c)']], constrained_layout=True, figsize=(8, 6), sharey=True)

for label, ax in axs.items():
    # label physical distance to the left and up:
    trans = mtransforms.ScaledTranslation(0.98, -5.2, fig.dpi_scale_trans)
    ax.text(0.0, 1.0, label, transform=ax.transAxes + trans,
            fontsize=10, va='bottom')

get_violin_plot(data_plot, axs['(a)'], 'All runs of Hyperband', f1=13, f2=12)
get_violin_plot1(data_plot_best_run, axs['(b)'], 'Best run', f1=13, f2=12)
get_violin_plot1(data_plot_worst_run, axs['(c)'], 'Worst run', f1=13, f2=12)
fig.tight_layout()
plt.show()
# plt.clf()
# plt.savefig("hyperband_results.pdf", dpi=600)


Figure 9 | Violin Plot for All Hyperband runs with different important hyperparameters for which priors are learned from configurations that achieve good performances

In [ ]:
data_plot_6 = df[df['imp_hyperparams'] == '[6]']['difference'].tolist()
data_plot_61 = df[df['imp_hyperparams'] == '[6, 1]']['difference'].tolist()
data_plot_614 = df[df['imp_hyperparams'] == '[6, 1, 4]']['difference'].tolist()
data_plot_6149 = df[df['imp_hyperparams'] == '[6, 1, 4, 9]']['difference'].tolist()

In [ ]:
fig, axs = plt.subplot_mosaic([['(a)', '(b)', '(c)', '(d)']], constrained_layout=True, figsize=(12, 6), sharey=True)

for label, ax in axs.items():
    # label physical distance to the left and up:
    trans = mtransforms.ScaledTranslation(1.2, -5.2, fig.dpi_scale_trans)
    ax.text(0.0, 1.0, label, transform=ax.transAxes + trans,
            fontsize=10, va='bottom')

get_violin_plot(data_plot_6, axs['(a)'], '{lr}', f1=11, f2=12)
get_violin_plot1(data_plot_61, axs['(b)'], '{lr, depth}', f1=11, f2=12)
get_violin_plot1(data_plot_614, axs['(c)'], '{lr, depth, activation}', f1=11, f2=12)
get_violin_plot1(data_plot_6149, axs['(d)'], '{lr, depth, activation, reuploading}', f1=11, f2=12)
fig.tight_layout()
plt.show()
# plt.clf()
# plt.savefig("hyperband_imp_hyperparams.pdf", dpi=600)